In [ ]:
!hostnamectl

In [ ]:
%cd /home/ir739wb/ilyarekun/nn_DeiT_25/

In [ ]:
%pwd

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torchvision.models import regnet_y_16gf, RegNet_Y_16GF_Weights
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
import random

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,garbage_collection_threshold:0.8"

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

# Проверка доступных GPU
import subprocess
subprocess.run(["nvidia-smi", "-L"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}, GPU count: {torch.cuda.device_count()}")

data_path = "/home/ir739wb/ilyarekun/nn_DeiT_25/cifar100_imagenet_style"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.15),
    transforms.RandomVerticalFlip(p=0.15),
    transforms.RandomRotation(30), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), 
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

train_dataset = datasets.ImageFolder(os.path.join(data_path, "train"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_path, "val"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=8)

# Загружаем предобученную модель RegNetY-16GF
teacher_model = models.regnet_y_16gf(weights=RegNet_Y_16GF_Weights.IMAGENET1K_V1)

# Модифицируем последний слой (fc) на Sequential с Dropout и Linear
in_features = teacher_model.fc.in_features
teacher_model.fc = nn.Sequential(
    nn.Linear(in_features, 100),
    nn.Dropout(p=0.6),
    nn.Linear(100, 100)
)

""" # Замораживаем все параметры модели
for param in teacher_model.parameters():
    param.requires_grad = False

# Размораживаем последние слои: например, trunk_output.block4 и fc
for param in teacher_model.trunk_output.block4.parameters():
    param.requires_grad = True
for param in teacher_model.fc.parameters():
    param.requires_grad = True """

# Переносим модель на устройство
teacher_model = teacher_model.to(device)

# Используем DataParallel, если доступно несколько GPU
if torch.cuda.device_count() > 1:
    print("Using DataParallel for multi-GPU training")
    teacher_model = nn.DataParallel(teacher_model)

# Оптимизатор: оптимизируем только размороженные параметры
optimizer = optim.Adam(teacher_model.parameters(), lr=0.00005, weight_decay=1e-2)
criterion = nn.CrossEntropyLoss()

# Планировщик скорости обучения
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.7)

train_losses = []
val_losses = []
val_accuracies = []
val_precisions = []
val_recalls = []
val_f1s = []
all_preds = []
all_labels = []

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        accuracy = 100 * correct / total
        val_accuracies.append(accuracy)
        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
        val_precisions.append(precision)
        val_recalls.append(recall)
        val_f1s.append(f1)
        
        str_res=f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {accuracy:.2f}%, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}"
        print(str_res)
        with open('results.txt', 'a') as f:
            f.write(str_res)
        
        scheduler.step()  # Обновление скорости обучения
        
train_model(teacher_model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100)

torch.save(teacher_model.state_dict(), 'teacher_regnety160_cifar100.pth')
#torch.save({'model': teacher_model.state_dict()}, 'teacher_regnety160_cifar100.pth')

print("Model weights saved to 'teacher_regnety160_cifar100.pth'")

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-680b5566-f66c-da58-c59b-1a9e054d807f)
GPU 1: NVIDIA A100-SXM4-40GB (UUID: GPU-72420bd6-f473-9783-fc6a-5894ccc331f2)
Using device: cuda, GPU count: 2
Using DataParallel for multi-GPU training
Epoch 1/100, Train Loss: 3.6672, Val Loss: 2.3850, Val Acc: 43.48%, Precision: 0.4380, Recall: 0.4348, F1: 0.3982
Epoch 2/100, Train Loss: 2.3699, Val Loss: 1.8381, Val Acc: 54.76%, Precision: 0.5002, Recall: 0.4912, F1: 0.4704
Epoch 3/100, Train Loss: 1.9601, Val Loss: 1.6842, Val Acc: 57.00%, Precision: 0.5177, Recall: 0.5174, F1: 0.5027
Epoch 4/100, Train Loss: 1.7377, Val Loss: 1.5841, Val Acc: 59.39%, Precision: 0.5361, Recall: 0.5366, F1: 0.5250
Epoch 5/100, Train Loss: 1.6196, Val Loss: 1.5828, Val Acc: 60.39%, Precision: 0.5504, Recall: 0.5500, F1: 0.5404
Epoch 6/100, Train Loss: 1.5101, Val Loss: 1.5166, Val Acc: 61.52%, Precision: 0.5591, Recall: 0.5609, F1: 0.5528
Epoch 7/100, Train Loss: 1.4173, Val Loss: 1.5540, Val Acc: 60.74%, Precision: 0.5650

In [ ]:
%ls

In [ ]:
print(all_labels)

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
epochs = np.arange(1, len(train_losses) + 1)

plt.figure()
plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, val_losses, label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epoch")
plt.legend()
plt.savefig("loss_vs_epoch.png")
plt.show()

plt.figure()
plt.plot(epochs, val_accuracies, label='Val Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Validation Accuracy vs Epoch")
plt.legend()
plt.savefig("accuracy_vs_epoch.png")
plt.show()

plt.figure()
plt.plot(epochs, val_precisions, label='Precision')
plt.plot(epochs, val_recalls, label='Recall')
plt.plot(epochs, val_f1s, label='F1 Score')
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("Precision, Recall, F1 vs Epoch")
plt.legend()
plt.savefig("metrics_vs_epoch.png")
plt.show()

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.show()


In [ ]:
import subprocess
subprocess.run(["nvidia-smi", "-L"])

In [ ]:
""" 0, NVIDIA A100-SXM4-40GB, GPU-04d2a6bb-264f-6b50-a917-1effa8949810
1, NVIDIA A100-SXM4-40GB, GPU-ff3ba929-da03-8441-6ab6-0ad20ecbac5f
2, NVIDIA A100-SXM4-40GB, GPU-a3c4b7d3-2308-0714-9328-ffe9888b564c
3, NVIDIA A100-SXM4-40GB, GPU-680b5566-f66c-da58-c59b-1a9e054d807f
4, NVIDIA A100-SXM4-40GB, GPU-72420bd6-f473-9783-fc6a-5894ccc331f2
5, NVIDIA A100-SXM4-40GB, GPU-2beb933a-6d58-ad8e-f216-de8e19ff4ecc
6, NVIDIA A100-SXM4-40GB, GPU-e000602e-d7c7-889e-303d-774fc518edb2
7, NVIDIA A100-SXM4-40GB, GPU-5c73ff1d-c5d0-f175-0241-c80dc2a3b970
(myenv) ir739wb@DGX:~/ilyarekun/nn_DeiT_25$  """

In [ ]:
""" import torch

# Путь к файлу
checkpoint_path = "/home/ir739wb/ilyarekun/nn_DeiT_25/cifar100_imagenet_style/teacher_regnety160_cifar100.pth"

# Загружаем контрольную точку
checkpoint = torch.load(checkpoint_path, map_location='cpu')

# Проверяем, что содержится в контрольной точке
print("Ключи в контрольной точке:", checkpoint.keys())

# Предполагаем, что 'model' — ключ для state_dict (может отличаться, проверьте вывод выше)
state_dict = checkpoint if 'model' not in checkpoint else checkpoint['model']

# Выводим первые несколько ключей state_dict
print("\nПервые 10 ключей в state_dict:")
for i, key in enumerate(state_dict.keys()):
    if i < 10:
        print(f"  {key}")

# Проверяем последний слой (обычно это 'head.fc.weight' или 'fc.weight')
for key in state_dict.keys():
    if 'fc.weight' in key or 'linear.weight' in key:
        num_classes = state_dict[key].shape[0]
        print(f"\nНайден последний слой: {key}")
        print(f"Количество классов: {num_classes}")
        break
else:
    print("\nПоследний слой не найден (возможно, нестандартное имя).")

# Выводим общее количество параметров
total_params = sum(p.numel() for p in state_dict.values())
print(f"\nОбщее количество параметров: {total_params}") """

In [ ]:
""" import torch
from collections import OrderedDict

# Путь к исходной контрольной точке
checkpoint_path = "/home/ir739wb/ilyarekun/nn_DeiT_25/cifar100_imagenet_style/teacher_regnety160_cifar100.pth"

# Загружаем контрольную точку
checkpoint = torch.load(checkpoint_path, map_location='cpu')
state_dict = checkpoint['model']

# Новый state_dict для адаптации
new_state_dict = OrderedDict()

# Параметры для адаптации
new_num_classes = 100  # Укажите нужное количество классов
remove_prefix = "base_model."  # Префикс, который нужно убрать

# Обрабатываем каждый ключ
for key, value in state_dict.items():
    # Убираем префикс "base_model."
    new_key = key.replace(remove_prefix, "")
    
    # Адаптируем последний слой
    if new_key == "fc.weight":
        old_num_classes = value.shape[0]  # 100
        print(f"Старое количество классов: {old_num_classes}, Новое: {new_num_classes}")
        if old_num_classes != new_num_classes:
            # Создаём новый весовой тензор с нужным количеством классов
            new_weight = torch.randn(new_num_classes, value.shape[1])  # Случайная инициализация
            new_state_dict[new_key] = new_weight
            print(f"Заменён вес {new_key} с {value.shape} на {new_weight.shape}")
        else:
            new_state_dict[new_key] = value
    elif new_key == "fc.bias":
        old_num_classes = value.shape[0]  # 100
        if old_num_classes != new_num_classes:
            # Создаём новый bias с нужным количеством классов
            new_bias = torch.randn(new_num_classes)  # Случайная инициализация
            new_state_dict[new_key] = new_bias
            print(f"Заменён bias {new_key} с {value.shape} на {new_bias.shape}")
        else:
            new_state_dict[new_key] = value
    else:
        # Копируем остальные параметры без изменений
        new_state_dict[new_key] = value

# Обновляем контрольную точку
checkpoint['model'] = new_state_dict

# Сохраняем адаптированную контрольную точку
new_checkpoint_path = "/home/ir739wb/ilyarekun/nn_DeiT_25/cifar100_imagenet_style/teacher_regnety160_adapted.pth"
torch.save(checkpoint, new_checkpoint_path)
print(f"Адаптированная контрольная точка сохранена в: {new_checkpoint_path}")

# Проверка структуры нового state_dict
print("\nПервые 10 ключей в новом state_dict:")
for i, key in enumerate(new_state_dict.keys()):
    if i < 10:
        print(f"  {key}") """